# Find The Regul Parameter
This notebook shows the example of finding the regul parameter in ppxf.  
Actually, it calculates a series of SFH, then store them into files.  
The chi-square statistic is also here.  

In [ ]:
import os
import sys
import math
import glob
import numpy as np
from os import path
import pandas as pd
import multiprocessing
from ppxf.ppxf import ppxf
from astropy.wcs import WCS
from astropy.io import fits
import ppxf as ppxf_package
import ppxf.ppxf_util as util
import ppxf.miles_util as lib
from astropy import units as u
import matplotlib.pyplot as plt
from multiprocessing import Pool
from specutils import Spectrum1D
from astropy import constants as con
from time import perf_counter as clock
from specutils.manipulation import FluxConservingResampler

In [ ]:
i=122

In [ ]:
def DER_SNR(flux):
    from numpy import array,where,median,abs 
    flux=array(flux)
    flux=array(flux[where(flux!=0.0)])
    n=len(flux)
    if (n>4):
        signal=median(flux)
        noise=0.6052697*median(abs(2.0*flux[2:n-2]-flux[0:n-4]-flux[4:n]))
        return signal,noise
    else:return 0,100


In [ ]:
def logSpecConverter(spec):
    waveOld=spec[:,0]*u.AA
    flux=spec[:,1]*u.erg/u.s/u.cm**2/u.AA
    noise=spec[:,2]*u.erg/u.s/u.cm**2/u.AA
    befSpec=Spectrum1D(spectral_axis=waveOld,flux=flux)
    fluxCon=FluxConservingResampler()
    waveNew=10**np.linspace(np.log10(waveOld.min().value),np.log10(waveOld.max().value),num=len(waveOld))*u.AA
    aftSpec=fluxCon(befSpec,waveNew)
    wave=aftSpec.wavelength.value
    flux=aftSpec.flux.value
    flux[np.isnan(flux)]=0
    
    befSpecN=Spectrum1D(spectral_axis=waveOld,flux=noise)
    fluxCon=FluxConservingResampler()
    aftSpecN=fluxCon(befSpecN,waveNew)
    noise=aftSpecN.flux.value
    noise[np.isnan(noise)]=9*10**9
    
    return np.array([wave,flux,noise]).T
def logSpecConvBack(wave,flux,waveOld):
    wave=wave*u.AA
    befSpec=Spectrum1D(spectral_axis=wave,flux=flux*u.erg/u.s/u.cm**2/u.AA)
    fluxCon=FluxConservingResampler()
    aftSpec=fluxCon(befSpec,waveOld*u.AA)
    wave=aftSpec.wavelength.value
    flux=aftSpec.flux.value
    flux[np.isnan(flux)]=0
    return flux

In [ ]:
FinalSelMuse=pd.read_csv('../MuseObject/FinalSelMuse.csv',index_col=0)

In [ ]:
oldName=FinalSelMuse['ArcName'][i]
newName=oldName.replace('ADP','CDP')
mucu=fits.open('../MuseObject/MuseResample/'+newName+'.fits')
wavestep=float(mucu[1].header['CD3_3'])
wavestart=float(mucu[1].header['CRVAL3'])
waveBase=np.arange(wavestart,mucu[1].shape[0]*wavestep+wavestart,wavestep)
waveBase=waveBase/(1+FinalSelMuse['Redshift'][i])
fitFlux=mucu[1].data*np.nan
gasFlux=mucu[1].data*np.nan
agePopu=np.zeros([50,7,mucu[1].data.shape[1],mucu[1].data.shape[2]])*np.nan
chi2Map=np.zeros([mucu[1].data.shape[1],mucu[1].data.shape[2]])
SPmask=np.load('../MuseMasker/FinalMask2/'+str(i)+'_'+FinalSelMuse['SN Name'][i]+'.npy')

In [ ]:
coordX=[]
coordY=[]
snrList=[]
for x in range(mucu[1].data.shape[1]):
    for y in range(mucu[1].data.shape[2]):
        if SPmask[x,y]==False:continue
        flux=mucu[1].data[:,x,y]
        flux=flux[np.isnan(flux)==False]
        if len(flux)<mucu[1].data.shape[0]/2:continue
        signal,noise=DER_SNR(flux)
        snrList.append(signal/noise)
        coordX.append(x)
        coordY.append(y)
coordX=np.array(coordX)
coordY=np.array(coordY)
snrList=np.array(snrList)
snrMax=np.argmax(snrList)

In [ ]:
def ppxfRunner(spec,regul):
    t=spec.copy()
    tie_balmer=False
    limit_doublets=False
    ppxf_dir=path.dirname(path.realpath(ppxf_package.__file__))
    z=0.0
    flux=t[:,1]
    galaxy=flux/np.median(flux)
    wave=t[:,0]
    wave*=np.median(util.vac_to_air(wave)/wave)
    noise=t[:,2]
    c=299792.458  # speed of light in km/s
    velscale=c*np.log(wave[1]/wave[0])  # eq.(8) of Cappellari (2017)
    FWHM_gal=2.26  # SDSS has an approximate instrumental resolution FWHM of 2.76A.#Califa is 6

    pathname='/scratch/user/chenxingzhuo/SNHZ/ppxfSFH/ppxfGrid/EMILES_PADOVA00_BASE_KB_FITS_Sigma228/*'
    #pathname='/scratch/user/chenxingzhuo/SNHZ/ppxfSFH/ppxfGrid/bc03_Miles_Kroupa/*'
    
    miles=lib.miles(pathname,velscale,FWHM_gal)
    reg_dim = miles.templates.shape[1:]
    stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)
    lam_range_gal = np.array([np.min(wave),np.max(wave)])/(1 + z)
    gas_templates,gas_names,line_wave=util.emission_lines(
        miles.log_lam_temp,lam_range_gal,FWHM_gal,
        tie_balmer=tie_balmer,limit_doublets=limit_doublets)
    templates=np.column_stack([stars_templates,gas_templates])

    dv=c*(miles.log_lam_temp[0]-np.log(wave[0]))  # eq.(8) of Cappellari (2017)
    vel=c*np.log(1+z)   # eq.(8) of Cappellari (2017)
    start=[vel,280.]  # (km/s), starting guess for [V, sigma]

    n_temps = stars_templates.shape[1]
    n_forbidden = np.sum(["[" in a for a in gas_names])  # forbidden lines contain "[*]"
    n_balmer = len(gas_names) - n_forbidden

    component = [0]*n_temps + [1]*n_balmer + [2]*n_forbidden
    gas_component = np.array(component) > 0  # gas_component=True for gas templates
    
    bounds_star=[[-500,500],[-300,300],[-0.1,0.1],[-0.1,0.1]]
    bounds_fbd=[[-500,500],[-300,300]]
    bounds_gas=[[-500,500],[-300,300]]
    bounds=[bounds_star,bounds_fbd,bounds_gas]
    
    moments = [4,2,2]
    start = [start, start, start]
    gas_reddening = 0 if tie_balmer else None

    pp = ppxf(templates, galaxy, noise, velscale, start,
              plot=True, moments=moments, degree=-1, mdegree=10, vsyst=dv,
              lam=wave, clean=False, regul=regul, reg_dim=reg_dim,# I pet regul=1/1.3 is the best
              component=component, gas_component=gas_component,
              gas_names=gas_names, gas_reddening=gas_reddening,bounds=bounds)
    weights=pp.weights[~gas_component]  # Exclude weights of the gas templates
    weights=weights.reshape(reg_dim)/weights.sum()  # Normalized
    return pp,weights

In [ ]:
xpix,ypix=coordX[snrMax],coordY[snrMax]
mask=np.isnan(mucu[1].data[:,xpix,ypix])
assert np.sum(mask==0)>len(mask)/2
flux=mucu[1].data[:,xpix,ypix]
ferr=mucu[2].data[:,xpix,ypix]
flux=flux[mask==0]
wave=waveBase[mask==0]
ferr=ferr[mask==0]**0.5
spec=np.array([wave,flux,ferr]).T
waveOld=spec[:,0]
spec=logSpecConverter(spec)
waveNew=spec[:,0]

In [ ]:
fitFluxList=[]
gasFluxList=[]
chi2List=[]
weightList=[]
for regul in [1024,512,256,128,64,32,16,8,4,2,1,1/2,1/4,1/8,1/16,1/32,1/64,1/128,1/256,0]:
    pp,weights=ppxfRunner(spec,regul=regul)
    chi2=pp.chi2
    fitFluxOnePix=pp.bestfit
    gasFluxOnePix=pp.gas_bestfit
    fitFluxOnePix=logSpecConvBack(waveNew,fitFluxOnePix,waveOld)
    gasFluxOnePix=logSpecConvBack(waveNew,gasFluxOnePix,waveOld)
    
    fitFluxList.append(fitFluxOnePix)
    gasFluxList.append(gasFluxOnePix)
    chi2List.append(chi2)
    weightList.append(weights)


In [ ]:
fitFluxList=np.array(fitFluxList)
gasFluxList=np.array(gasFluxList)
chi2List=np.array(chi2List)
weightList=np.array(weightList)
np.save('RegulFind/'+FinalSelMuse['SN Name'][i]+'.fitFlux.npy',fitFluxList)
np.save('RegulFind/'+FinalSelMuse['SN Name'][i]+'.gasFlux.npy',gasFluxList)
np.save('RegulFind/'+FinalSelMuse['SN Name'][i]+'.weights.npy',weightList)
np.save('RegulFind/'+FinalSelMuse['SN Name'][i]+'.chi2.npy',chi2List)

# The Chi-Square Statistics
This is the code finding the smoothed SFH among all the searched regul parameters.  

In [ ]:
i=122

In [ ]:
def specMaxFinder(i):
    oldName=FinalSelMuse['ArcName'][i]
    newName=oldName.replace('ADP','CDP')
    mucu=fits.open('../MuseObject/MuseResample/'+newName+'.fits')
    wavestep=float(mucu[1].header['CD3_3'])
    wavestart=float(mucu[1].header['CRVAL3'])
    waveBase=np.arange(wavestart,mucu[1].shape[0]*wavestep+wavestart,wavestep)
    waveBase=waveBase/(1+FinalSelMuse['Redshift'][i])
    SPmask=np.load('../MuseMasker/FinalMask2/'+str(i)+'_'+FinalSelMuse['SN Name'][i]+'.npy')

    coordX=[]
    coordY=[]
    snrList=[]
    for x in range(mucu[1].data.shape[1]):
        for y in range(mucu[1].data.shape[2]):
            if SPmask[x,y]==False:continue
            flux=mucu[1].data[:,x,y]
            flux=flux[np.isnan(flux)==False]
            if len(flux)<mucu[1].data.shape[0]/2:continue
            signal,noise=DER_SNR(flux)
            snrList.append(signal/noise)
            coordX.append(x)
            coordY.append(y)
    coordX=np.array(coordX)
    coordY=np.array(coordY)
    snrList=np.array(snrList)
    snrMax=np.argmax(snrList)


    xpix,ypix=coordX[snrMax],coordY[snrMax]
    mask=np.isnan(mucu[1].data[:,xpix,ypix])
    assert np.sum(mask==0)>len(mask)/2
    flux=mucu[1].data[:,xpix,ypix]
    ferr=mucu[2].data[:,xpix,ypix]
    flux=flux[mask==0]
    wave=waveBase[mask==0]
    ferr=ferr[mask==0]**0.5
    spec=np.array([wave,flux,ferr]).T
    return spec

In [ ]:
fitFluxList=np.load('RegulFind/'+FinalSelMuse['SN Name'][i]+'.fitFlux.npy')
gasFluxList=np.load('RegulFind/'+FinalSelMuse['SN Name'][i]+'.gasFlux.npy')
weightList=np.load('RegulFind/'+FinalSelMuse['SN Name'][i]+'.weights.npy')
chi2List=np.load('RegulFind/'+FinalSelMuse['SN Name'][i]+'.chi2.npy')

weightListFlat=weightList.sum(axis=2)
avgAge=(weightListFlat*np.log10(agestep)).sum(axis=1)
chi2List[np.abs(avgAge-avgAge[-1])>0.3]=9*10**99
lenFlux=len(fitFluxList[-1])
dchiList=(chi2List-chi2List[-1])/chi2List[-1]*lenFlux
regu=np.argmin(np.abs(dchiList-(2*lenFlux)**0.5))
if regu>=19:regu=18
reguChos=regu
regu=regulList[regu]
RegulList[i]=regu

spec=specMaxFinder(i)
spec[:,1]=spec[:,1]/np.nanmedian(spec[:,1])

reguSelList=[0,5,10,15,19]
for j,reguSel in enumerate(reguSelList):
    plt.subplot(2,6,j+1)
    plt.plot(spec[:,0],spec[:,1])
    plt.plot(spec[:,0],fitFluxList[reguSel])
    plt.title('Regul:'+str(reguSel)+' '+str(regulList[reguSel]))
    plt.subplot(2,6,j+7)
    miles.plot(weightList[reguSel])
plt.subplot(2,6,6)
plt.plot(spec[:,0],spec[:,1])
plt.plot(spec[:,0],fitFluxList[reguSel])
plt.title('Regul Chosen:'+str(reguChos)+' '+str(regulList[reguChos]))
plt.subplot(2,6,12)
miles.plot(weightList[reguChos])
plt.gcf().set_size_inches(30,10)
#plt.savefig('plotout/plotMd4/'+str(i)+'_'+FinalSelMuse['SN Name'][i]+'.png')


In [ ]:
ls ../MuseRegulFind

In [ ]:
ls -alht

In [ ]:
!du -h GroupMap